In [1]:
import pandas as pd
df = pd.read_csv("../isec_train.csv", index_col=0)
df

,/JS,/AA,/OpenAction,/AcroForm,/RichMedia,/Launch,/EmbeddedFile,/XFA,/URI,/F,...,mkdir,replace,fromCharCode,getAnnots,ip,xref_size,trailer_size,file_size,nof_stream,class
file_000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,3,21,21,67.784,5,0
file_001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,5,98,174,14.627,23,0
file_002,2,0,1,0,0,0,0,0,0,0,...,0,0,1,1,1,10,10,16.013,8,1
file_003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,3,21,21,63.609,5,0
file_004,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,4,29,37,83.376,35,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
file_495,2,0,1,0,0,0,0,0,0,0,...,0,0,1,1,1,10,10,8.181,8,1
file_496,2,0,1,0,0,0,0,0,0,0,...,0,0,0,1,1,11,11,8.118,9,1
file_497,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,4,25,33,78.263,28,0
file_498,2,0,1,0,0,0,0,0,0,0,...,0,0,0,1,1,11,11,8.157,9,1


- minmax scaling

In [8]:
from sklearn.preprocessing import MinMaxScaler
import joblib
minmax_scaler = MinMaxScaler()
minmax_scaler.fit(df.iloc[:, :-1])
minmax_scaled = minmax_scaler.fit_transform(df.iloc[:, :-1])
df_scaled = pd.DataFrame(minmax_scaled, index=df.index, columns=df.columns[:-1])
print(list(df_scaled.columns))
df_scaled['class'] = df['class']
joblib.dump(minmax_scaler, "../scaler.save") 
df_scaled.to_csv("../isec_train_scaled.csv")

['/JS', '/AA', '/OpenAction', '/AcroForm', '/RichMedia', '/Launch', '/EmbeddedFile', '/XFA', '/URI', '/F', '/GoToE', '/Extends', '/Annot', '/Length', '/Filter', '/FlateDecode', '/XObject', '/Encrypt', '/CreationDate', '/Author', 'subject', '/ModDate', 'html', 'Script', 'powershell.exe', 'cmd.exe', 'ascii_c', 'sys', 'root', 'reg', 'exe', 'http', 'domain', 'filepath', 'ascii', 'xml', 'dll', 'unescape', '%u9090%u9090', 'getIcon', 'temptmp', 'eval', '%appdata%', 'chunk', 'base64', 'mkdir', 'replace', 'fromCharCode', 'getAnnots', 'ip', 'xref_size', 'trailer_size', 'file_size', 'nof_stream']


#model import
- n_estimators : 학습 모델의 수, 많아질수록 성능 향상의 가능성이 있으나, 속도가 느려짐
- learning_rate : 학습률, 너무 크면 gradient 발산의 가능성이 있으며, 너무 작으면 학습이 느림
- max_depth : 최대 탐색 깊이, 너무 크면 과적합의 가능성, 너무 작으면 학습 성능 저하
- min_samples_split : 분할 종료 최소 샘플 수, 큰 수면 과적합을 막지만 학습 성능 저하 가능성
- min_samples_leaf : leaf node가 되기 위한 최소 샘플 수, min_samples_split과 비슷한 용도
- random_state : 특정 숫자로 고정 시에 실행 시마다 고정된 결과 반환

In [4]:
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=500, learning_rate=0.2, max_depth=4, random_state = 32)

In [8]:
X = df_scaled.iloc[:, :-1]
y = df_scaled.iloc[:, -1]

In [9]:
print(X)
print(y)

               /JS  /AA  /OpenAction  /AcroForm  /RichMedia  /Launch  \
file_000  0.000000  0.0          0.0        0.0         0.0      0.0   
file_001  0.000000  0.0          0.0        0.0         0.0      0.0   
file_002  0.666667  0.0          1.0        0.0         0.0      0.0   
file_003  0.000000  0.0          0.0        0.0         0.0      0.0   
file_004  0.000000  0.0          0.0        1.0         0.0      0.0   
...            ...  ...          ...        ...         ...      ...   
file_495  0.666667  0.0          1.0        0.0         0.0      0.0   
file_496  0.666667  0.0          1.0        0.0         0.0      0.0   
file_497  0.000000  0.0          0.0        1.0         0.0      0.0   
file_498  0.666667  0.0          1.0        0.0         0.0      0.0   
file_499  0.000000  0.0          0.0        1.0         0.0      0.0   

          /EmbeddedFile  /XFA  /URI   /F  ...  base64  mkdir  replace  \
file_000            0.0   0.0   0.0  0.0  ...     0.0    0.0  

In [10]:
# 모델 학습
model.fit(X, y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=None,
              num_parallel_tree=None, random_state=32, ...)

In [12]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(X)

In [13]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
def get_clf_eval(y_test, y_pred):
    confusion = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred)
    AUC = roc_auc_score(y_test, y_pred)
    print('오차행렬:\n', confusion)
    print('\n정확도: {:.4f}'.format(accuracy))
    print('정밀도: {:.4f}'.format(precision))
    print('재현율: {:.4f}'.format(recall))
    print('F1: {:.4f}'.format(F1))
    print('AUC: {:.4f}'.format(AUC))


In [14]:
get_clf_eval(y, y_pred)

오차행렬:
 [[250   0]
 [  0 250]]

정확도: 1.0000
정밀도: 1.0000
재현율: 1.0000
F1: 1.0000
AUC: 1.0000
